In [1]:
!pip install pymysql


In [8]:
import os
import pandas as pd
import pymysql
import sys

In [9]:
df=pd.read_csv("D:/datasets/orcamento/2022_orcamento.csv",nrows=10000,encoding='latin-1',delimiter=';')


In [24]:
df.head(15)

,EXERCÍCIO,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO SUBORDINADO,NOME ÓRGÃO SUBORDINADO,CÓDIGO UNIDADE ORÇAMENTÁRIA,NOME UNIDADE ORÇAMENTÁRIA,CÓDIGO FUNÇÃO,NOME FUNÇÃO,CÓDIGO SUBFUNÇÃO,...,CÓDIGO CATEGORIA ECONÔMICA,NOME CATEGORIA ECONÔMICA,CÓDIGO GRUPO DE DESPESA,NOME GRUPO DE DESPESA,CÓDIGO ELEMENTO DE DESPESA,NOME ELEMENTO DE DESPESA,ORÇAMENTO INICIAL (R$),ORÇAMENTO ATUALIZADO (R$),ORÇAMENTO EMPENHADO (R$),ORÇAMENTO REALIZADO (R$)
0,2022,20000,Presidência da República,20101,Presidência da República,20101,PRESIDENCIA DA REPUBLICA,4,Administração,122,...,4,DESPESAS DE CAPITAL,4,Investimentos,51,Obras e Instalações,"5797530,99","5797530,99","5797530,99","1755185,23"
1,2022,20000,Presidência da República,20101,Presidência da República,20101,PRESIDENCIA DA REPUBLICA,4,Administração,122,...,3,DESPESAS CORRENTES,3,Outras Despesas Correntes,30,Material de Consumo,"1446919,74","1446919,74","1446919,74","1446919,74"
2,2022,20000,Presidência da República,20101,Presidência da República,20101,PRESIDENCIA DA REPUBLICA,4,Administração,122,...,3,DESPESAS CORRENTES,3,Outras Despesas Correntes,40,Serviços de Tecnologia da Informação e Comunic...,"168630,78","168630,78","168630,78","0,00"
3,2022,20000,Presidência da República,20101,Presidência da República,20101,PRESIDENCIA DA REPUBLICA,4,Administração,122,...,3,DESPESAS CORRENTES,3,Outras Despesas Correntes,0,Não informado,"2144449,48","21903,48","0,00","0,00"
4,2022,20000,Presidência da República,20101,Presidência da República,20101,PRESIDENCIA DA REPUBLICA,4,Administração,122,...,4,DESPESAS DE CAPITAL,4,Investimentos,0,Não informado,"30248301,79","66831,79","0,00","0,00"
5,2022,20000,Presidência da República,20101,Presidência da República,20101,PRESIDENCIA DA REPUBLICA,4,Administração,122,...,4,DESPESAS DE CAPITAL,4,Investimentos,52,Equipamentos e Material Permanente,"10664874,22","10664874,22","10664874,22","7011502,68"
6,2022,20000,Presidência da República,20101,Presidência da República,20101,PRESIDENCIA DA REPUBLICA,4,Administração,122,...,4,DESPESAS DE CAPITAL,4,Investimentos,39,Outros Serviços de Terceiros - Pessoa Jurídica,"200933,00","200933,00","200933,00","15210,00"
7,2022,20000,Presidência da República,20101,Presidência da República,20118,AGENCIA BRASILEIRA DE INTELIGENCIA - ABIN,6,Segurança pública,301,...,3,DESPESAS CORRENTES,3,Outras Despesas Correntes,96,Ressarcimento de Despesas de Pessoal Requisitado,"163884,19","163884,19","163884,19","89891,58"
8,2022,20000,Presidência da República,20101,Presidência da República,20118,AGENCIA BRASILEIRA DE INTELIGENCIA - ABIN,6,Segurança pública,301,...,3,DESPESAS CORRENTES,3,Outras Despesas Correntes,0,Não informado,"804887,00","304887,00","0,00","0,00"
9,2022,20000,Presidência da República,20101,Presidência da República,20118,AGENCIA BRASILEIRA DE INTELIGENCIA - ABIN,6,Segurança pública,301,...,3,DESPESAS CORRENTES,3,Outras Despesas Correntes,39,Outros Serviços de Terceiros - Pessoa Jurídica,"1978086,30","1978086,30","1978086,30","1902448,23"


In [11]:
df.columns

Index(['EXERCÍCIO', 'CÓDIGO ÓRGÃO SUPERIOR', 'NOME ÓRGÃO SUPERIOR',
       'CÓDIGO ÓRGÃO SUBORDINADO', 'NOME ÓRGÃO SUBORDINADO',
       'CÓDIGO UNIDADE ORÇAMENTÁRIA', 'NOME UNIDADE ORÇAMENTÁRIA',
       'CÓDIGO FUNÇÃO', 'NOME FUNÇÃO', 'CÓDIGO SUBFUNÇÃO', 'NOME SUBFUNÇÃO',
       'CÓDIGO PROGRAMA ORÇAMENTÁRIO', 'NOME PROGRAMA ORÇAMENTÁRIO',
       'CÓDIGO AÇÃO', 'NOME AÇÃO', 'CÓDIGO CATEGORIA ECONÔMICA',
       'NOME CATEGORIA ECONÔMICA', 'CÓDIGO GRUPO DE DESPESA',
       'NOME GRUPO DE DESPESA', 'CÓDIGO ELEMENTO DE DESPESA',
       'NOME ELEMENTO DE DESPESA', 'ORÇAMENTO INICIAL (R$)',
       'ORÇAMENTO ATUALIZADO (R$)', 'ORÇAMENTO EMPENHADO (R$)',
       'ORÇAMENTO REALIZADO (R$)'],
      dtype='object')

In [23]:
def processing_data(data, number=10000):
    data="D:/datasets/orcamento/2022_orcamento.csv"
    if not os.path.isfile(data):
        print("Dataset not found. Please, add dataset to Data folder")
        exit()

    # reading just 100000 rows the dataframe
    df = pd.read_csv(data, nrows=10000, encoding="latin-1", sep=';')

    return df

# Carregando data para o dw
def load_data_dimension(dataset):
    # Connect to the database
    connection = pymysql.connect(host='localhost', user='guilherme', password='bibiboboB1!', db='schemaz', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

    try:
        with connection.cursor() as cursor:
            for index, row in dataset.iterrows():
                # Adding data to Dim_despesa
                nm_categoria_economica = row['NOME CATEGORIA ECONÔMICA']
                nm_grupo_despesa = row['NOME GRUPO DE DESPESA']
                nm_elemento_despesa = row['NOME ELEMENTO DE DESPESA']

                sql = """INSERT INTO dim_despesa (nm_categoria_economica, nm_grupo_despesa, nm_elemento_despesa) VALUES (%s, %s, %s)"""
                cursor.execute(sql, (nm_categoria_economica, nm_grupo_despesa, nm_elemento_despesa))

                # Adding data to dim_funcao
                nm_funcao = row['NOME FUNÇÃO']
                nm_sub_funcao = row['NOME SUBFUNÇÃO']

                sql = """INSERT INTO dim_funcao (nm_funcao, nm_sub_funcao) VALUES (%s, %s)"""
                cursor.execute(sql, (nm_funcao, nm_sub_funcao))

                # Adding data to dim_orcamento
                nm_unid_orcamentaria = row['NOME UNIDADE ORÇAMENTÁRIA']
                nm_programa_orcamentario = row['NOME PROGRAMA ORÇAMENTÁRIO']
                nm_acao_orcamentaria = row['NOME AÇÃO']

                sql = """INSERT INTO dim_orcamentos (nm_unid_orcamentaria, nm_programa_orcamentario, nm_acao_orcamentaria) VALUES (%s, %s, %s)"""
                cursor.execute(sql, (nm_unid_orcamentaria, nm_programa_orcamentario, nm_acao_orcamentaria))

                # Adding data to dim_orgao
                nm_orgao_superior = row['NOME ÓRGÃO SUPERIOR']
                nm_orgao_subordinado = row['NOME ÓRGÃO SUBORDINADO']

                sql = """INSERT INTO dim_orgao (nm_orgao_superior, nm_orgao_subordinado) VALUES (%s, %s) """
                cursor.execute(sql, (nm_orgao_superior, nm_orgao_subordinado))

        connection.commit()      
    finally:
        
        connection.close()
        
        
def load_data_fact(dataset):
    # Connect to the database
    connection = pymysql.connect(host='localhost', user='guilherme', password='bibiboboB1!', db='schemaz', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

    try:
        with connection.cursor() as cursor:
            # Iterating over rows
            for index, row in dataset.iterrows():
                # Adding data to tb_fato
                nmr_orcamento_atualizado = row['ORÇAMENTO ATUALIZADO (R$)']
                nmr_orcamento_realizado = row['ORÇAMENTO REALIZADO (R$)']

                # Incrementing ID for each row
                id_incremental = int(index) + 1

                # Loading data to fact table
                sql_fact = """INSERT INTO tb_fato (nmr_orcamento_realizado, nmr_orcamento_atualizado, id_funcao, id_orgao, id_despesa, id_orcamento) VALUES (%s, %s, %s, %s, %s, %s)"""
                cursor.execute(sql_fact, (nmr_orcamento_realizado, nmr_orcamento_atualizado, id_incremental, id_incremental, id_incremental, id_incremental))

        # Commit data
        connection.commit()

    finally:
        # Close connection
        connection.close()





df= processing_data("D:/datasets/orcamento/2022_orcamento.csv", 10000)
load_data_dimension(df)
load_data_fact(df)